In [ ]:
!pip install ultralytics
!pip install filterpy
!pip install scipy
!pip install opencv-python
!pip install matplotlib

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import os

In [ ]:
print("Please upload your tennis video file:")
uploaded = files.upload()

# Get the uploaded file name
video_filename = list(uploaded.keys())[0]
print(f"Uploaded file: {video_filename}")

In [ ]:
"""
Fixed Tennis Player Tracking Solution for Google Colab
Resolves PyTorch/Dynamo circular import issues
"""

# Cell 1: Clean installation and setup
!pip uninstall torch torchvision torchaudio -y
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install ultralytics==8.0.196
!pip install filterpy
!pip install scipy
!pip install opencv-python-headless
!pip install matplotlib
!pip install numpy

# Restart runtime after installation
import os
os.kill(os.getpid(), 9)

In [ ]:
"""
Final Fixed Tennis Player Tracking Solution
Addresses PyTorch 2.7 weights_only loading issue
"""

import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
from collections import defaultdict
import os
from IPython.display import Video, display
import warnings
import torch
warnings.filterwarnings('ignore')

# Fix PyTorch weights loading issue
torch.serialization.add_safe_globals([
    'ultralytics.nn.tasks.DetectionModel',
    'ultralytics.nn.modules.head.Detect',
    'ultralytics.nn.modules.conv.Conv',
    'ultralytics.nn.modules.block.C2f',
    'ultralytics.nn.modules.block.Bottleneck',
    'ultralytics.nn.modules.block.SPPF',
    'torch.nn.modules.upsampling.Upsample',
    'torch.nn.modules.pooling.MaxPool2d',
    'torch.nn.modules.activation.SiLU'
])

print("🔧 PyTorch safe globals configured for YOLO")

# Set environment variables
os.environ['PYTORCH_DISABLE_DYNAMO'] = '1'
os.environ['TORCH_DYNAMO_DISABLE'] = '1'

# Import ultralytics with proper error handling
try:
    from ultralytics import YOLO
    print("✅ YOLO imported successfully")
except ImportError as e:
    print(f"❌ YOLO import error: {e}")
    !pip install ultralytics==8.0.196
    from ultralytics import YOLO

try:
    from scipy.optimize import linear_sum_assignment
    print("✅ SciPy imported successfully")
except ImportError:
    print("Installing scipy...")
    !pip install scipy
    from scipy.optimize import linear_sum_assignment

class CentroidTracker:
    """
    Robust centroid-based tracker for tennis players
    Uses distance-based matching and handles disappearances
    """

    def __init__(self, max_disappeared=20, max_distance=100):
        self.next_object_id = 0
        self.objects = {}
        self.disappeared = {}
        self.max_disappeared = max_disappeared
        self.max_distance = max_distance

    def register(self, centroid):
        """Register a new object with next available ID"""
        self.objects[self.next_object_id] = centroid
        self.disappeared[self.next_object_id] = 0
        self.next_object_id += 1

    def deregister(self, object_id):
        """Remove an object from tracking"""
        del self.objects[object_id]
        del self.disappeared[object_id]

    def update(self, input_centroids):
        """Update tracker with new centroids"""
        # If no input centroids, mark all as disappeared
        if len(input_centroids) == 0:
            for object_id in list(self.disappeared.keys()):
                self.disappeared[object_id] += 1
                if self.disappeared[object_id] > self.max_disappeared:
                    self.deregister(object_id)
            return self.objects

        # If no existing objects, register all input centroids
        if len(self.objects) == 0:
            for centroid in input_centroids:
                self.register(centroid)
        else:
            # Match existing objects to new centroids
            object_centroids = list(self.objects.values())
            object_ids = list(self.objects.keys())

            # Compute distance matrix
            D = np.linalg.norm(np.array(object_centroids)[:, np.newaxis] - input_centroids, axis=2)

            # Find optimal assignment using Hungarian algorithm
            if D.size > 0:
                try:
                    row_indices, col_indices = linear_sum_assignment(D)
                except:
                    # Fallback to simple nearest neighbor
                    row_indices = D.min(axis=1).argsort()
                    col_indices = D.argmin(axis=1)[row_indices]

                # Track used indices
                used_row_indices = set()
                used_col_indices = set()

                # Update matched objects
                for (row, col) in zip(row_indices, col_indices):
                    if (row in used_row_indices or col in used_col_indices or
                        row >= len(object_ids) or col >= len(input_centroids)):
                        continue

                    if D[row, col] <= self.max_distance:
                        object_id = object_ids[row]
                        self.objects[object_id] = input_centroids[col]
                        self.disappeared[object_id] = 0
                        used_row_indices.add(row)
                        used_col_indices.add(col)

                # Handle unmatched objects and detections
                unused_row_indices = set(range(len(object_ids))).difference(used_row_indices)
                unused_col_indices = set(range(len(input_centroids))).difference(used_col_indices)

                # Mark unmatched objects as disappeared
                for row in unused_row_indices:
                    if row < len(object_ids):
                        object_id = object_ids[row]
                        self.disappeared[object_id] += 1
                        if self.disappeared[object_id] > self.max_disappeared:
                            self.deregister(object_id)

                # Register new objects
                for col in unused_col_indices:
                    if col < len(input_centroids):
                        self.register(input_centroids[col])

        return self.objects

class TennisPlayerTracker:
    """
    Enhanced Tennis Player Tracker with robust detection and tracking
    """

    def __init__(self, video_path):
        self.video_path = video_path
        print("🔄 Loading YOLO model...")

        # Load YOLO with proper error handling and weights_only fix
        try:
            # Create a custom loading function that sets weights_only=False
            def load_yolo_model():
                # Temporarily patch torch.load to use weights_only=False
                original_load = torch.load

                def patched_load(*args, **kwargs):
                    kwargs['weights_only'] = False
                    return original_load(*args, **kwargs)

                torch.load = patched_load
                try:
                    model = YOLO('yolov8n.pt')
                    model.overrides['verbose'] = False
                    return model
                finally:
                    torch.load = original_load

            self.model = load_yolo_model()
            print("✅ YOLO model loaded successfully!")

        except Exception as e:
            print(f"❌ Error loading YOLO: {e}")
            print("🔄 Trying alternative loading method...")

            # Alternative: Download and load manually
            try:
                import urllib.request
                import os

                weights_path = 'yolov8n.pt'
                if not os.path.exists(weights_path):
                    print("📥 Downloading YOLOv8 weights...")
                    urllib.request.urlretrieve(
                        'https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt',
                        weights_path
                    )

                # Load with weights_only=False
                torch.serialization.add_safe_globals(['ultralytics.nn.tasks.DetectionModel'])
                self.model = YOLO(weights_path)
                self.model.overrides['verbose'] = False
                print("✅ YOLO model loaded with alternative method!")

            except Exception as e2:
                print(f"❌ Alternative loading also failed: {e2}")
                raise Exception("Could not load YOLO model. Please check your internet connection.")

        self.tracker = CentroidTracker(max_disappeared=15, max_distance=80)
        self.player_distances = defaultdict(float)
        self.player_paths = defaultdict(list)
        self.player_bboxes = defaultdict(list)
        self.frame_count = 0

        # Detection parameters optimized for tennis
        self.confidence_threshold = 0.3
        self.min_bbox_area = 1500
        self.max_bbox_area = 50000

    def detect_players(self, frame):
        """
        Enhanced player detection with better filtering
        """
        try:
            # Run YOLO detection
            results = self.model.predict(
                frame,
                classes=[0],  # person class
                conf=self.confidence_threshold,
                verbose=False,
                device='cpu'  # Force CPU to avoid GPU issues
            )

            detections = []
            bboxes = []

            for result in results:
                if result.boxes is not None and len(result.boxes) > 0:
                    boxes = result.boxes.cpu().numpy()

                    for box in boxes:
                        try:
                            x1, y1, x2, y2 = box.xyxy[0]
                            confidence = float(box.conf[0])

                            # Calculate bbox properties
                            width = x2 - x1
                            height = y2 - y1
                            area = width * height
                            aspect_ratio = height / width if width > 0 else 0

                            # Enhanced filtering for tennis players
                            if (self.min_bbox_area < area < self.max_bbox_area and
                                width > 25 and height > 50 and
                                1.0 < aspect_ratio < 5.0 and
                                confidence > self.confidence_threshold):

                                center_x = (x1 + x2) / 2
                                center_y = (y1 + y2) / 2

                                detections.append([center_x, center_y])
                                bboxes.append([x1, y1, x2, y2, confidence])

                        except Exception as box_error:
                            continue  # Skip problematic boxes

            return np.array(detections), bboxes

        except Exception as e:
            print(f"⚠️  Detection error in frame {self.frame_count}: {e}")
            return np.array([]), []

    def calculate_distance(self, pos1, pos2):
        """Calculate Euclidean distance between two positions"""
        return math.sqrt((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)

    def pixels_to_meters(self, pixel_distance):
        """Convert pixel distance to estimated meters"""
        # Tennis court dimensions: 23.77m long, 8.23m wide (singles)
        # Estimate scale based on typical video resolution
        estimated_court_length_pixels = 800
        real_court_length = 23.77
        scale_factor = real_court_length / estimated_court_length_pixels
        return pixel_distance * scale_factor

    def draw_tracking_info(self, frame, object_id, centroid, bbox_info=None):
        """Enhanced drawing with better visibility"""
        x, y = int(centroid[0]), int(centroid[1])

        # Distinct colors for different players
        colors = [
            (0, 255, 0),    # Green
            (255, 0, 0),    # Blue
            (0, 0, 255),    # Red
            (255, 255, 0),  # Cyan
            (255, 0, 255),  # Magenta
            (0, 255, 255)   # Yellow
        ]
        color = colors[object_id % len(colors)]

        # Draw bounding box if available
        if bbox_info and len(bbox_info) >= 4:
            x1, y1, x2, y2 = bbox_info[:4]
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 3)

        # Draw center point with outline
        cv2.circle(frame, (x, y), 12, (255, 255, 255), -1)  # White background
        cv2.circle(frame, (x, y), 10, color, -1)             # Colored center
        cv2.circle(frame, (x, y), 12, (0, 0, 0), 2)          # Black outline

        # Prepare text info
        distance = self.player_distances[object_id]
        distance_meters = self.pixels_to_meters(distance)

        player_label = f"Player {object_id}"
        distance_label = f"{distance:.0f}px"
        meter_label = f"{distance_meters:.1f}m"

        # Calculate text positions
        text_y_start = max(y - 60, 20)

        # Draw text background
        texts = [player_label, distance_label, meter_label]
        max_width = 0
        for text in texts:
            text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
            max_width = max(max_width, text_size[0])

        # Background rectangle
        cv2.rectangle(frame,
                     (x - 5, text_y_start - 5),
                     (x + max_width + 10, text_y_start + 60),
                     (0, 0, 0), -1)
        cv2.rectangle(frame,
                     (x - 5, text_y_start - 5),
                     (x + max_width + 10, text_y_start + 60),
                     color, 2)

        # Draw text
        cv2.putText(frame, player_label, (x, text_y_start + 15),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.putText(frame, distance_label, (x, text_y_start + 35),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        cv2.putText(frame, meter_label, (x, text_y_start + 50),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        # Draw movement trail
        if len(self.player_paths[object_id]) > 1:
            points = np.array(self.player_paths[object_id][-20:], dtype=np.int32)
            if len(points) > 1:
                # Draw trail with decreasing opacity
                for i in range(1, len(points)):
                    alpha = i / len(points)
                    thickness = max(1, int(3 * alpha))
                    cv2.line(frame, tuple(points[i-1]), tuple(points[i]), color, thickness)

    def process_video(self, output_path, process_every_n_frames=1):
        """
        Process video with optimizations for large files
        """
        cap = cv2.VideoCapture(self.video_path)

        # Get video properties
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        print(f"📹 Video Properties:")
        print(f"   Resolution: {width}x{height}")
        print(f"   FPS: {fps}")
        print(f"   Total frames: {total_frames}")
        print(f"   Duration: {total_frames/fps:.2f} seconds")
        print(f"   Processing every {process_every_n_frames} frame(s)")

        # Setup video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        print(f"\n🎬 Starting video processing...")

        frame_skip_counter = 0

        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                self.frame_count += 1
                frame_skip_counter += 1

                # Process detection only every N frames for speed
                if frame_skip_counter >= process_every_n_frames:
                    frame_skip_counter = 0

                    # Detect players
                    centroids, bboxes = self.detect_players(frame)

                    # Update tracker
                    objects = self.tracker.update(centroids)
                else:
                    # For skipped frames, just use previous tracking results
                    objects = self.tracker.objects
                    bboxes = []

                # Process tracked objects
                for object_id, centroid in objects.items():
                    # Find corresponding bbox if available
                    bbox_info = None
                    if len(bboxes) > 0 and len(centroids) > 0:
                        distances_to_bboxes = []
                        for bbox in bboxes:
                            bbox_center = [(bbox[0] + bbox[2])/2, (bbox[1] + bbox[3])/2]
                            dist = self.calculate_distance(centroid, bbox_center)
                            distances_to_bboxes.append(dist)

                        if distances_to_bboxes:
                            closest_idx = np.argmin(distances_to_bboxes)
                            if distances_to_bboxes[closest_idx] < 60:
                                bbox_info = bboxes[closest_idx]

                    # Update path and calculate distance
                    self.player_paths[object_id].append(centroid)

                    if len(self.player_paths[object_id]) > 1:
                        prev_pos = self.player_paths[object_id][-2]
                        distance = self.calculate_distance(prev_pos, centroid)
                        # Scale distance by frame skip to maintain accuracy
                        self.player_distances[object_id] += distance * process_every_n_frames

                    # Draw tracking info
                    self.draw_tracking_info(frame, object_id, centroid, bbox_info)

                # Add frame information
                active_players = len(objects)
                progress = (self.frame_count / total_frames) * 100

                info_text = f"Frame: {self.frame_count}/{total_frames} ({progress:.1f}%)"
                players_text = f"Players: {active_players}"

                # Draw info background
                cv2.rectangle(frame, (10, 10), (500, 70), (0, 0, 0), -1)
                cv2.rectangle(frame, (10, 10), (500, 70), (255, 255, 255), 2)

                # Draw info text
                cv2.putText(frame, info_text, (20, 35),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(frame, players_text, (20, 55),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

                # Write frame to output
                out.write(frame)

                # Progress updates
                if self.frame_count % 1000 == 0:
                    print(f"⏳ Processed {self.frame_count}/{total_frames} frames ({progress:.1f}%)")
                    print(f"   Currently tracking {active_players} player(s)")

        except Exception as e:
            print(f"❌ Error during video processing: {e}")

        finally:
            cap.release()
            out.release()
            cv2.destroyAllWindows()

        print(f"✅ Video processing completed!")
        return dict(self.player_distances)

def create_comprehensive_analysis(distances, paths):
    """Create detailed analysis and visualization"""
    if not distances:
        print("❌ No tracking data available for analysis")
        return

    # Create figure with subplots
    fig = plt.figure(figsize=(20, 15))
    gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

    fig.suptitle('🎾 COMPREHENSIVE TENNIS PLAYER TRACKING ANALYSIS 🎾',
                 fontsize=20, fontweight='bold', y=0.95)

    players = list(distances.keys())
    pixel_distances = list(distances.values())
    meter_distances = [d * 23.77 / 800 for d in pixel_distances]

    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8', '#F7DC6F']

    # 1. Distance comparison (pixels)
    ax1 = fig.add_subplot(gs[0, 0])
    bars1 = ax1.bar([f'P{p}' for p in players], pixel_distances,
                   color=[colors[i % len(colors)] for i in range(len(players))])
    ax1.set_title('Total Distance (Pixels)', fontweight='bold', fontsize=14)
    ax1.set_ylabel('Distance (pixels)')
    ax1.grid(True, alpha=0.3)

    for bar, value in zip(bars1, pixel_distances):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + max(pixel_distances)*0.01,
                f'{value:.0f}', ha='center', va='bottom', fontweight='bold')

    # 2. Distance comparison (meters)
    ax2 = fig.add_subplot(gs[0, 1])
    bars2 = ax2.bar([f'P{p}' for p in players], meter_distances,
                   color=[colors[i % len(colors)] for i in range(len(players))])
    ax2.set_title('Total Distance (Meters)', fontweight='bold', fontsize=14)
    ax2.set_ylabel('Distance (meters)')
    ax2.grid(True, alpha=0.3)

    for bar, value in zip(bars2, meter_distances):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height + max(meter_distances)*0.01,
                f'{value:.1f}m', ha='center', va='bottom', fontweight='bold')

    # 3. Pie chart
    ax3 = fig.add_subplot(gs[0, 2])
    if len(pixel_distances) > 1:
        wedges, texts, autotexts = ax3.pie(pixel_distances,
                                          labels=[f'Player {p}' for p in players],
                                          autopct='%1.1f%%',
                                          colors=[colors[i % len(colors)] for i in range(len(players))],
                                          startangle=90)
        ax3.set_title('Distance Distribution', fontweight='bold', fontsize=14)

    # 4. Movement paths
    ax4 = fig.add_subplot(gs[1, :])
    ax4.set_title('Player Movement Paths on Court', fontweight='bold', fontsize=16)
    ax4.set_xlabel('X Position (pixels)')
    ax4.set_ylabel('Y Position (pixels)')

    for i, (player_id, path) in enumerate(paths.items()):
        if len(path) > 1:
            path_array = np.array(path)
            color = colors[i % len(colors)]

            # Plot movement path
            ax4.plot(path_array[:, 0], path_array[:, 1],
                    color=color, linewidth=3, alpha=0.8, label=f'Player {player_id}')

            # Mark start and end positions
            ax4.scatter(path_array[0, 0], path_array[0, 1],
                       color=color, s=200, marker='o',
                       edgecolors='white', linewidth=2, label=f'Start P{player_id}')
            ax4.scatter(path_array[-1, 0], path_array[-1, 1],
                       color=color, s=200, marker='s',
                       edgecolors='white', linewidth=2, label=f'End P{player_id}')

            # Add annotations
            ax4.annotate(f'START\nP{player_id}',
                        (path_array[0, 0], path_array[0, 1]),
                        xytext=(10, 10), textcoords='offset points',
                        bbox=dict(boxstyle='round,pad=0.3', facecolor=color, alpha=0.7),
                        fontsize=8, fontweight='bold', color='white')

    ax4.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax4.grid(True, alpha=0.3)
    ax4.invert_yaxis()

    # 5. Statistics table
    ax5 = fig.add_subplot(gs[2, :2])
    ax5.axis('off')
    ax5.set_title('Detailed Player Statistics', fontweight='bold', fontsize=16, pad=20)

    # Prepare table data
    table_data = [['Player', 'Distance\n(Pixels)', 'Distance\n(Meters)', 'Tracking\nPoints',
                   'Avg Speed\n(px/frame)', 'Court Coverage\n(X-axis)', 'Court Coverage\n(Y-axis)']]

    for i, player_id in enumerate(players):
        path = paths[player_id]
        tracking_points = len(path)
        avg_speed = pixel_distances[i] / tracking_points if tracking_points > 0 else 0

        # Calculate court coverage
        if len(path) > 0:
            path_array = np.array(path)
            x_coverage = np.max(path_array[:, 0]) - np.min(path_array[:, 0])
            y_coverage = np.max(path_array[:, 1]) - np.min(path_array[:, 1])
        else:
            x_coverage = y_coverage = 0

        table_data.append([
            f'Player {player_id}',
            f'{pixel_distances[i]:.0f}',
            f'{meter_distances[i]:.2f}',
            f'{tracking_points}',
            f'{avg_speed:.2f}',
            f'{x_coverage:.0f}px',
            f'{y_coverage:.0f}px'
        ])

    # Create table
    table = ax5.table(cellText=table_data[1:], colLabels=table_data[0],
                     cellLoc='center', loc='center', bbox=[0, 0, 1, 1])
    table.auto_set_font_size(False)
    table.set_fontsize(11)
    table.scale(1, 2.5)

    # Style table
    for i in range(len(table_data)):
        for j in range(len(table_data[0])):
            cell = table[(i, j)]
            if i == 0:  # Header
                cell.set_facecolor('#2E86AB')
                cell.set_text_props(weight='bold', color='white')
            else:  # Data rows
                player_idx = (i - 1) % len(colors)
                cell.set_facecolor(colors[player_idx])
                cell.set_text_props(weight='bold', color='white')

    # 6. Performance summary
    ax6 = fig.add_subplot(gs[2, 2])
    ax6.axis('off')
    ax6.set_title('Analysis Summary', fontweight='bold', fontsize=14)

    total_distance_px = sum(pixel_distances)
    total_distance_m = sum(meter_distances)
    avg_distance_px = total_distance_px / len(pixel_distances) if pixel_distances else 0
    avg_distance_m = total_distance_m / len(meter_distances) if meter_distances else 0

    summary_text = f"""
🏃 Players Tracked: {len(players)}
📏 Total Distance: {total_distance_px:.0f} pixels
📐 Total Distance: {total_distance_m:.1f} meters
📊 Average per Player: {avg_distance_px:.0f} px
📊 Average per Player: {avg_distance_m:.1f} m

🎯 Most Active: Player {players[np.argmax(pixel_distances)]}
📈 Distance: {max(pixel_distances):.0f} px
"""

    ax6.text(0.1, 0.9, summary_text, transform=ax6.transAxes, fontsize=12,
             verticalalignment='top', bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))

    plt.tight_layout()
    plt.show()

# MAIN EXECUTION FUNCTION
def run_tennis_tracking_ultimate():
    """
    Ultimate tennis tracking function with all fixes applied
    """
    print("🎾 ULTIMATE TENNIS PLAYER TRACKING SYSTEM 🎾")
    print("=" * 80)

    video_path = "tennis_video_assignment.mp4"

    if not os.path.exists(video_path):
        print(f"❌ Video file '{video_path}' not found!")
        print("📁 Current directory contents:")
        for file in os.listdir('.'):
            print(f"   - {file}")
        return None, None

    print(f"✅ Found video file: {video_path}")

    try:
        # Initialize tracker
        tracker = TennisPlayerTracker(video_path)

        # Process video
        output_path = "tracked_tennis_ultimate.mp4"
        print(f"\n🎬 Processing video with enhanced tracking...")

        # For faster processing, you can increase process_every_n_frames
        # process_every_n_frames=2 means process every 2nd frame (2x faster)
        final_distances = tracker.process_video(output_path, process_every_n_frames=2)

        print(f"\n🏆 TRACKING ANALYSIS COMPLETE! 🏆")
        print("=" * 60)

        if final_distances:
            print(f"\n📊 Successfully tracked {len(final_distances)} players:")

            for player_id, distance in final_distances.items():
                meter_distance = tracker.pixels_to_meters(distance)
                tracking_points = len(tracker.player_paths[player_id])
                avg_speed = distance / tracking_points if tracking_points > 0 else 0

                print(f"\n🏃 Player {player_id}:")
                print(f"   📏 Total distance: {distance:.0f} pixels ({meter_distance:.2f} meters)")
                print(f"   🎯 Tracking points: {tracking_points}")
                print(f"   ⚡ Average speed: {avg_speed:.2f} pixels/frame")

            print(f"\n🎥 Output video saved as: {output_path}")

            # Create comprehensive analysis
            print(f"\n📈 Generating comprehensive analysis...")
            create_comprehensive_analysis(final_distances, tracker.player_paths)

            # Display video
            try:
                print(f"\n🎬 Displaying output video...")
                display(Video(output_path, width=900, height=600))
            except Exception as e:
                print(f"💡 Video saved as '{output_path}' - download to view")
                print(f"   Display error: {e}")

            return tracker, final_distances

        else:
            print("❌ No players were successfully tracked")
            print("\nPossible solutions:")
            print("- Check video quality and lighting")
            print("- Ensure players are clearly visible")
            print("- Try adjusting confidence threshold")
            return None, None

    except Exception as e:
        print(f"❌ Fatal error: {e}")
        import traceback
        traceback.print_exc()
        return None, None

# Execute the ultimate tracking system
print("🚀 Launching Ultimate Tennis Tracking System...")
tracker_result, distances_result = run_tennis_tracking_ultimate()

if tracker_result and distances_result:
    print("\n🎉 SUCCESS! Complete analysis generated above.")
    print("📁 Files created:")
    print("   - tracked_tennis_ultimate.mp4 (output video)")
    print("   - Comprehensive analysis charts")

    # Additional insights
    print(f"\n🔍 FINAL INSIGHTS:")
    total_players = len(distances_result)
    total_distance = sum(distances_result.values())
    print(f"   🏃 Players detected: {total_players}")
    print(f"   📏 Combined distance: {total_distance:.0f} pixels")
    print(f"   📐 Combined distance: {total_distance * 23.77 / 800:.1f} meters (estimated)")
    print(f"   🎯 Video duration: {tracker_result.frame_count / 119:.1f} seconds")

else:
    print("\n❌ Tracking unsuccessful. Please check error messages above.")